In [1]:
#import
import  numpy as np
import pygame, sys
import time

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# euclidian dist check
def dist(p1,p2):
    dx=p2[0]-p1[0]
    dy=p2[1]-p1[1]
    return np.sqrt(dx*dx+dy*dy)

In [3]:
#obj model

class Obj:
    def __init__(self,x,y,color) -> None:
        self.x=x
        self.y=y
        self.color=color
        self.reservedRobot=None
        self.finished=False
    def getPos(self):
        return(self.x, self.y)
    def draw(self,screen):
        r=10
        pygame.draw.ellipse(screen, self.color,[self.x-r,self.y-r,2*r,2*r],2)

In [4]:
# robot model

class Robot:
    def __init__(self, x, y):
        self.x=x
        self.y=y
        self.attachedObj=None
        self.target=None

    def getPos(self):
        return (self.x, self.y)

    def draw(self, screen):
        r=20
        pygame.draw.ellipse(screen, (255, 0, 0),[self.x-r, self.y-r, 2*r, 2*r], 2)

    def simulate(self):
        if self.target !=None:
            p1=self.target
            p2=(self.x, self.y)
            v=np.array(p1)-np.array(p2)
            d=dist(p1,p2)
            if d>0:
                v=v/d
            self.x+=v[0]*5
            self.y+=v[1]*5
        if self.attachedObj !=None:
            self.attachedObj.x=self.x
            self.attachedObj.y=self.y
    
    def findNearestObj(self,objs,threshhold=100500):
        res=None
        D=100500
        for o in objs:
            if o.reservedRobot !=None and o.reservedRobot !=self:
                continue
            if o.finished:
                continue
            dNew=dist(o.getPos(),self.getPos())
            if dNew<D:
                D=dNew
                res=o
        if D>threshhold:
            res=None
        return res
    
    def take(self,obj):
        if obj !=None:
            self.attachedObj=obj

In [5]:
# task distribution

def distributeTasks(robots, objs,goal):
    #distributing to every robot
    for r in robots:
        #finish check
        if r.attachedObj !=None and dist(r.getPos(), goal.getPos()) <20:
            r.attachedObj.finished=True
            r.attachedObj=None
            r.target=None
        else:
            #taking possibility check
            if r.attachedObj==None:
                obj=r.findNearestObj(objs)
                if obj !=None and dist(r.getPos(), obj.getPos()) < 20:
                    r.take(obj)
                    r.target = goal.getPos()
                    return
        #checking near objs
        if r.target==None and r.attachedObj==None:
            obj=r.findNearestObj(objs)
            if obj is None:
                r.target=goal.getPos()
                continue
            if obj.reservedRobot !=None:
                r.target=goal.getPos()
                continue
            r.target=obj.getPos()
            obj.reservedRobot=r

In [6]:
#finished alg criteria

def checkMission(robots, objs, goal):
    for r in robots:
        if dist(r.getPos(), goal.getPos())>20:
            return False
    for o in objs:
        if o.reservedRobot==None:
            return False
    return True
        

In [7]:
#obj generation sequence

def generateObjs(N):
    res=[]
    for i in range(N):
        o=Obj(np.random.randint(50, 1500-50),np.random.randint(50, 1300-50),(0,255,0))
        res.append(o)
    return res

In [8]:
#main
def main():

    #canvas parameters
    WIDTH=1500
    HEIGHT=1300

    pygame.init()
    screen=pygame.display.set_mode((WIDTH,HEIGHT))

    start=time.time()

    robots= [
        Robot(150,150),
        Robot(150,250),
        Robot(150,350),
        Robot(150,450),
        Robot(150,550),
        Robot(150,650)
            ]
    objs=generateObjs(5)

    goal=Obj(750,450,(0,0,255))


    while True:
        #interface
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                pygame.quit()
                sys.exit()
        #simulation
        if checkMission(robots,objs,goal):
                break

        screen.fill((255,255,255))

        distributeTasks(robots,objs,goal)

        for r in robots:
            r.simulate()
            r.draw(screen)

        for o in objs:
            o.draw(screen)

        goal.draw(screen)

        pygame.display.update()
        pygame.time.delay(50)

        # group task completion calculation
        end=time.time()
        print(end-start)
            

In [9]:
main()
NN=[1,2,3,4,5,6]
MM=[5,10,15]

0.05208730697631836
0.1030876636505127
0.15408754348754883
0.2040877342224121
0.25508880615234375
0.3060886859893799
0.3570892810821533
0.40808939933776855
0.4590904712677002
0.5100893974304199
0.5620920658111572
0.6130900382995605
0.6640901565551758
0.7150874137878418
0.766089916229248
0.8180913925170898
0.8690905570983887
0.9200928211212158
0.97109055519104
1.0220913887023926
1.0730915069580078
1.1240932941436768
1.1750915050506592
1.2260918617248535
1.277092695236206
1.3280935287475586
1.3790943622589111
1.4311017990112305
1.4820928573608398
1.5330936908721924
1.5840933322906494
1.6350934505462646
1.6860966682434082
1.7370944023132324
1.7880949974060059
1.8400962352752686
1.8930928707122803
1.9440951347351074
1.9950966835021973
2.0470988750457764
2.098095417022705
2.1490986347198486
2.199098825454712
2.250096321105957
2.3010969161987305
2.3520963191986084
2.4030966758728027
2.4550981521606445
2.5070972442626953
2.558095932006836
2.609098434448242
2.66009783744812
2.7110979557037354


: 